In [55]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [56]:
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Bangalore").text

In [57]:
soup = BeautifulSoup(data, 'html.parser')

In [58]:
neighborhoodList = []

In [59]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [60]:
neighborhoodList 

['►  Agara, Bangalore\u200e (2 C,  6 F)',
 '►  Arekere\u200e (5 F)',
 '►  Banashankari\u200e (1 C,  4 F)',
 '►  Banaswadi\u200e (2 F)',
 '►  Basavanagudi\u200e (5 C,  11 F)',
 '►  Begur, Bangalore\u200e (1 C,  6 F)',
 '►  Bellandur\u200e (1 C,  4 F)',
 '►  BEML\u200e (6 F)',
 '►  Bengaluru Pete\u200e (9 C,  4 F)',
 '►  Bidadi\u200e (2 C,  1 F)',
 '►  Bommasandra\u200e (33 F)',
 '►  Brigade Road, Bangalore\u200e (3 C,  8 F)',
 '►  Chandapura\u200e (4 F)',
 '►  Devanahalli\u200e (4 C,  12 F)',
 '►  Dhobi Ghat (Bangalore)\u200e (3 F)',
 '►  Domlur\u200e (5 C,  23 F)',
 '►  Electronics City\u200e (2 C,  34 F)',
 '►  Fraser Town, Bangalore\u200e (1 C,  10 F)',
 '►  Girinagar\u200e (1 C,  11 F)',
 '►  HBR Layout\u200e (1 C,  1 F)',
 '►  Hebbal\u200e (2 C,  3 F)',
 '►  Hosakote\u200e (1 C)',
 '►  HSR Layout\u200e (1 C,  12 F)',
 '►  Indiranagar\u200e (4 C,  8 F)',
 '►  Ittamadu\u200e (3 F)',
 '►  J. P. Nagar\u200e (2 C)',
 '►  Jakkur\u200e (2 C,  1 F)',
 '►  Jayanagar, Bangalore\u200e (1 C,  

In [61]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df.head()

,Neighborhood
0,"► Agara, Bangalore‎ (2 C, 6 F)"
1,► Arekere‎ (5 F)
2,"► Banashankari‎ (1 C, 4 F)"
3,► Banaswadi‎ (2 F)
4,"► Basavanagudi‎ (5 C, 11 F)"


In [62]:
df.shape

(59, 1)

In [63]:
df['Neighborhood'] =  df['Neighborhood'].apply(lambda x: x.replace('►','').replace("Bangalore","").replace(", ","")) 

In [64]:
df.head()

,Neighborhood
0,Agara‎ (2 C 6 F)
1,Arekere‎ (5 F)
2,Banashankari‎ (1 C 4 F)
3,Banaswadi‎ (2 F)
4,Basavanagudi‎ (5 C 11 F)


In [65]:

# Importing re package for using regular expressions 
import re 
  
# Function to clean the names 
def Clean_names(City_name): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times 
    if re.search('\(.*', City_name): 
  
        # Extract the position of beginning of pattern 
        pos = re.search('\(.*', City_name).start() 
  
        # return the cleaned name 
        return City_name[:pos] 
  
    else: 
        # if clean up needed return the same name 
        return City_name 
          
# Updated the city columns 
df['Neighborhood'] = df['Neighborhood'].apply(Clean_names) 
  
# Print the updated dataframe 
print(df)


                Neighborhood
0                    Agara‎ 
1                  Arekere‎ 
2             Banashankari‎ 
3                Banaswadi‎ 
4             Basavanagudi‎ 
5                    Begur‎ 
6                Bellandur‎ 
7                     BEML‎ 
8           Bengaluru Pete‎ 
9                   Bidadi‎ 
10             Bommasandra‎ 
11            Brigade Road‎ 
12              Chandapura‎ 
13             Devanahalli‎ 
14               Dhobi Ghat 
15                  Domlur‎ 
16        Electronics City‎ 
17             Fraser Town‎ 
18               Girinagar‎ 
19              HBR Layout‎ 
20                  Hebbal‎ 
21                Hosakote‎ 
22              HSR Layout‎ 
23             Indiranagar‎ 
24                Ittamadu‎ 
25             J. P. Nagar‎ 
26                  Jakkur‎ 
27               Jayanagar‎ 
28       Jeevanbheemanagar‎ 
29              Kettohalli‎ 
30               Kodihalli‎ 
31             Konanakunte‎ 
32             Koramangala‎ 
33         Kri

In [66]:
df.head()

,Neighborhood
0,Agara‎
1,Arekere‎
2,Banashankari‎
3,Banaswadi‎
4,Basavanagudi‎


In [67]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [68]:
import geocoder
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [69]:
coords

[[12.842830000000049, 77.48759000000007],
 [12.885680000000036, 77.59668000000005],
 [12.87341248765075, 77.53801254302039],
 [13.028477106077231, 77.63188968370176],
 [12.938980000000072, 77.57137000000006],
 [12.882450000000063, 77.62475000000006],
 [12.927350000000047, 77.67185000000006],
 [12.964900000000057, 77.59240000000005],
 [12.966180000000065, 77.58690000000007],
 [12.966180000000065, 77.58690000000007],
 [12.817530000000033, 77.67879000000005],
 [12.966235877087087, 77.6067910877087],
 [12.794010000000071, 77.70015000000006],
 [13.250110000000063, 77.70788000000005],
 [12.966180000000065, 77.58690000000007],
 [12.943290000000047, 77.65602000000007],
 [12.839884329698537, 77.67221065989045],
 [12.998940000000061, 77.61276000000004],
 [12.947163401610013, 77.55479203719199],
 [13.041247426010807, 77.62417570944993],
 [13.049810000000036, 77.58903000000004],
 [13.077180000000055, 77.80178000000006],
 [12.977831148479487, 77.54380233014024],
 [12.978220000000022, 77.63397000000

In [70]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [71]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']


In [73]:
print(df.shape)
df

(59, 3)


,Neighborhood,Latitude,Longitude
0,Agara‎,12.842830,77.487590
1,Arekere‎,12.885680,77.596680
2,Banashankari‎,12.873412,77.538013
3,Banaswadi‎,13.028477,77.631890
4,Basavanagudi‎,12.938980,77.571370
5,Begur‎,12.882450,77.624750
6,Bellandur‎,12.927350,77.671850
7,BEML‎,12.964900,77.592400
8,Bengaluru Pete‎,12.966180,77.586900
9,Bidadi‎,12.966180,77.586900


In [75]:
df.to_csv("Bangalore.csv", index=False)

In [77]:
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [79]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)  
    
map

In [81]:
map.save('map.html')

In [82]:
CLIENT_ID = 'BDMA5HXBI2MZN5DRFUUAAJNZWYTHRWF0533KMFAVIE5ED4XN' # your Foursquare ID
CLIENT_SECRET = 'VZNLFVPDJDG1QWXP0SGNUBI4DY5BTBO4FUY3VM1UVUZQHHXO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: BDMA5HXBI2MZN5DRFUUAAJNZWYTHRWF0533KMFAVIE5ED4XN
CLIENT_SECRET:VZNLFVPDJDG1QWXP0SGNUBI4DY5BTBO4FUY3VM1UVUZQHHXO


In [84]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [85]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()


(3566, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Arekere‎,12.88568,77.59668,Decathlon Sports India Pvt Ltd,12.887513,77.597712,Sporting Goods Shop
1,Arekere‎,12.88568,77.59668,Cinepolis,12.876119,77.595455,Multiplex
2,Arekere‎,12.88568,77.59668,Swensens,12.876071,77.595542,Ice Cream Shop
3,Arekere‎,12.88568,77.59668,Natural Ice Cream,12.892188,77.598222,Ice Cream Shop
4,Arekere‎,12.88568,77.59668,Chavadi,12.892199,77.602538,BBQ Joint


In [86]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Arekere‎,76,76,76,76,76,76
BEML‎,100,100,100,100,100,100
Banashankari‎,7,7,7,7,7,7
Banaswadi‎,53,53,53,53,53,53
Basavanagudi‎,100,100,100,100,100,100
Begur‎,15,15,15,15,15,15
Bellandur‎,87,87,87,87,87,87
Bengaluru Pete‎,100,100,100,100,100,100
Bidadi‎,100,100,100,100,100,100


In [87]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 215 uniques categories.


In [88]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [89]:
# one hot encoding
bl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bl_onehot.columns[-1]] + list(bl_onehot.columns[:-1])
bl_onehot = bl_onehot[fixed_columns]

print(bl_onehot.shape)
bl_onehot.head()

(3566, 216)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Arekere‎,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arekere‎,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Arekere‎,0,0,0,0,0,0,0,0,0,0,0

In [90]:
bl_grouped = bl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(bl_grouped.shape)
bl_grouped

(58, 216)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Arekere‎,0.000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.013158,0.013158,0.00,0.013158,0.000000,0.00000,0.000000,0.013158,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.013158,0.013158,0.000000,0.000000,0.000000,0.000000,0.013158,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.078947,0.000000,0.00,0.00,0.000000,0.00,0.039474,0.00,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.013158,0.00000,0.000000,0.00,0.00,0.00000,0.052632,0.000000,0.000000,0.000000,0.013158,0.000000,0.026316,0.013158,0.013158,0.000000,0.000000,0.000000,0.039474,0.00,0.000000,0.000000,0.00000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.013158,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.105263

In [91]:
len(bl_grouped[bl_grouped["Shopping Mall"] > 0])

27

In [92]:
bl_mall = bl_grouped[["Neighborhoods","Shopping Mall"]]


In [94]:
bl_mall.head()


,Neighborhoods,Shopping Mall
0,Arekere‎,0.026316
1,BEML‎,0.010000
2,Banashankari‎,0.000000
3,Banaswadi‎,0.000000
4,Basavanagudi‎,0.000000


In [96]:
# set number of clusters
kclusters = 3

bl_clustering = bl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 0, 0, 0, 0, 2, 2, 2, 0], dtype=int32)

In [98]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bl_merged = bl_mall.copy()

# add clustering labels
bl_merged["Cluster Labels"] = kmeans.labels_

In [100]:
bl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Arekere‎,0.026316,1
1,BEML‎,0.010000,2
2,Banashankari‎,0.000000,0
3,Banaswadi‎,0.000000,0
4,Basavanagudi‎,0.000000,0


In [103]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bl_merged = bl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(bl_merged.shape)
bl_merged.head() # check the last columns!

(58, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Arekere‎,0.026316,1,12.885680,77.596680
1,BEML‎,0.010000,2,12.964900,77.592400
2,Banashankari‎,0.000000,0,12.873412,77.538013
3,Banaswadi‎,0.000000,0,13.028477,77.631890
4,Basavanagudi‎,0.000000,0,12.938980,77.571370


In [105]:
# sort the results by Cluster Labels
print(bl_merged.shape)
bl_merged.sort_values(["Cluster Labels"], inplace=True)
bl_merged


(58, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
46,Sahakara Nagar‎,0.000000,0,13.065275,77.600146
31,Koramangala‎,0.000000,0,12.920040,77.625460
30,Konanakunte‎,0.000000,0,12.882330,77.569260
29,Kodihalli‎,0.000000,0,12.966200,77.649820
56,Yeshwantpur‎,0.000000,0,13.039120,77.577970
50,Tippasandra‎,0.000000,0,12.971710,77.647800
26,Jayanagar‎,0.000000,0,12.928720,77.582810
25,Jakkur‎,0.000000,0,13.075640,77.603940
45,Ramamurthy Nagar‎,0.000000,0,13.023820,77.677850
22,Indiranagar‎,0.000000,0,12.978220,77.633970


In [107]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bl_merged['Latitude'], bl_merged['Longitude'], bl_merged['Neighborhood'], bl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [108]:
# save the map as HTML file
map_clusters.save('map_clusters.html')


In [111]:
bl_merged.loc[bl_merged['Cluster Labels'] == 0]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
46,Sahakara Nagar‎,0.0,0,13.065275,77.600146
31,Koramangala‎,0.0,0,12.920040,77.625460
30,Konanakunte‎,0.0,0,12.882330,77.569260
29,Kodihalli‎,0.0,0,12.966200,77.649820
56,Yeshwantpur‎,0.0,0,13.039120,77.577970
50,Tippasandra‎,0.0,0,12.971710,77.647800
26,Jayanagar‎,0.0,0,12.928720,77.582810
25,Jakkur‎,0.0,0,13.075640,77.603940
45,Ramamurthy Nagar‎,0.0,0,13.023820,77.677850
22,Indiranagar‎,0.0,0,12.978220,77.633970


In [112]:
bl_merged.loc[bl_merged['Cluster Labels'] == 1]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
36,Mahadevapura‎,0.020408,1,12.994090,77.666330
37,Majestic,0.020000,1,12.977590,77.572560
38,Malleswaram‎,0.020000,1,13.006322,77.568416
48,Shivajinagar‎,0.020000,1,12.987200,77.604010
0,Arekere‎,0.026316,1,12.885680,77.596680
33,Kundalahalli‎,0.024096,1,12.967520,77.715000
32,Krishnarajapura‎,0.029851,1,13.000390,77.683680
24,J. P. Nagar‎,0.020000,1,12.908310,77.590240
16,Fraser Town‎,0.021277,1,12.998940,77.612760
10,Brigade Road‎,0.020000,1,12.966236,77.606791


In [114]:
bl_merged.loc[bl_merged['Cluster Labels'] == 2]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
51,UB City‎,0.010000,2,12.971370,77.595820
28,Kettohalli‎,0.010000,2,12.966180,77.586900
39,Marathahalli‎,0.013333,2,12.954660,77.707520
34,Madiwala‎,0.010000,2,12.920520,77.620900
27,Jeevanbheemanagar‎,0.010000,2,12.966180,77.586900
13,Dhobi Ghat,0.010000,2,12.966180,77.586900
8,Bidadi‎,0.010000,2,12.966180,77.586900
7,Bengaluru Pete‎,0.010000,2,12.966180,77.586900
6,Bellandur‎,0.011494,2,12.927350,77.671850
1,BEML‎,0.010000,2,12.964900,77.592400
